In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import numpy as np

In [2]:
# Input shape
img_height, img_width = 227, 227
num_classes = 19

# Path
dataset_dir = 'Bacteria_Dataset_unzipped'

In [7]:
#Datagenerators
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

valid_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



Found 333 images belonging to 19 classes.
Found 76 images belonging to 19 classes.


In [8]:
# Learning rate scheduler
def lr_scheduler(epoch):
    return 0.0001 * 0.98**epoch

In [9]:
#ALexNet Model
model = Sequential()

# Layer 1: Convolutional and Max Pooling Layer
model.add(Conv2D(96,(11,11), strides=(4,4), activation = 'relu', input_shape = (img_height, img_width,3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))


# Layer 2: Convolutional and Max Pooling Layer
model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Layer 3: Convolutional Layer
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 4: Convolutional Layer
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 5: Convolutional and Max Pooling Layer
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Layer 6: Fully Connected Layer
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())

# Layer 7: Fully Connected Layer
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())

# layer 8: Output Layer
model.add(Dense(num_classes, activation='softmax'))


In [10]:
# Initialize a dictionary to store metrics
training_metrics = {'accuracy': [], 'loss': []}


In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate schedule and early stopping
lr_callback = LearningRateScheduler(lr_scheduler)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Trainining
history = model.fit(
    train_generator,
    epochs=50,  # You can adjust the number of epochs
    validation_data=valid_generator,
    callbacks=[lr_callback, early_stopping_callback]
)



Epoch 1/50
11/11 [==============================] - 85s 8s/step - loss: 2.3856 - accuracy: 0.3333 - val_loss: 124.3786 - val_accuracy: 0.0526 - lr: 1.0000e-04
Epoch 2/50
11/11 [==============================] - 76s 7s/step - loss: 1.5502 - accuracy: 0.5405 - val_loss: 199.4206 - val_accuracy: 0.0526 - lr: 9.8000e-05
Epoch 3/50
11/11 [==============================] - 70s 6s/step - loss: 1.5577 - accuracy: 0.5285 - val_loss: 277.6872 - val_accuracy: 0.0526 - lr: 9.6040e-05
Epoch 4/50
11/11 [==============================] - 79s 7s/step - loss: 1.1995 - accuracy: 0.6036 - val_loss: 172.4160 - val_accuracy: 0.0526 - lr: 9.4119e-05
Epoch 5/50
11/11 [==============================] - 70s 6s/step - loss: 1.1647 - accuracy: 0.6156 - val_loss: 113.7327 - val_accuracy: 0.0526 - lr: 9.2237e-05
Epoch 6/50
11/11 [==============================] - 76s 7s/step - loss: 1.1095 - accuracy: 0.6607 - val_loss: 134.7204 - val_accuracy: 0.0526 - lr: 9.0392e-05
Epoch 7/50
11/11 [============================

In [18]:
import json
# Add training metrics to the dictionary
training_metrics['accuracy'] += history.history['accuracy']
training_metrics['loss'] += history.history['loss']


# Save the metrics to a JSON file
with open('Models/AI_Alex_Net.json', 'w') as json_file:
    json.dump(training_metrics, json_file)

In [19]:
# Save the model
model.save('Models/AI_AlexNet.h5')

In [21]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import os
import random

# Input shape
img_height, img_width = 227, 227

# Path to the saved model
model_path = 'Models/AI_AlexNet.h5'

# Path to the new dataset
new_dataset_dir = 'Bacteria_dataset_unzipped'  # Replace with the actual path

# Load the saved model
model = load_model(model_path)

# Define a function to randomly select 100 images
def select_random_images(num_images_to_test, dataset_dir):
    random_images = []
    for class_folder in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_folder)
        if os.path.isdir(class_path):
            image_files = os.listdir(class_path)
            random_images += [os.path.join(class_path, img) for img in random.sample(image_files, num_images_to_test // len(os.listdir(dataset_dir)))]
    return random_images

# Load and preprocess 100 random images
num_images_to_test = 100
random_image_files = select_random_images(num_images_to_test, new_dataset_dir)

def preprocess_image(file_path):
    try:
        img = tf.io.read_file(file_path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize(img, [img_height, img_width])
        img = img / 255.0  # Normalize
        return img
    except Exception as e:
        print(f"Error processing image: {file_path}")
        return None

# Initialize counters for correct and wrong predictions
correct_predictions = 0
wrong_predictions = 0

# Get class indices from the model
class_indices = train_generator.class_indices  # Assuming you have the same class indices as before

# Reverse the mapping to get class names from class indices
class_names = {v: k for k, v in class_indices.items()}

# Perform inference on 100 random images
for img_path in random_image_files:
    img = preprocess_image(img_path)
    if img is not None:
        img = tf.expand_dims(img, 0)  # Add batch dimension
        predictions = model.predict(img)
        predicted_class_index = tf.argmax(predictions[0]).numpy()

        # Get the class name from class indices
        true_class_index = class_indices[os.path.basename(os.path.dirname(img_path))]
        true_class = class_names[true_class_index]

        # Print predicted and original class names
#         print(f"Predicted: {class_names[predicted_class_index]}, Original: {true_class}")

        if predicted_class_index == true_class_index:
            correct_predictions += 1
        else:
            wrong_predictions += 1

# Print the results
print(f"Correct Predictions: {correct_predictions}")
print(f"Wrong Predictions: {wrong_predictions}")


1/1 [==============================] - 0s 197ms/step
Predicted: Neisseria.gonorrhoeae, Original: Acinetobacter.baumanii
1/1 [==============================] - 0s 58ms/step
Predicted: Neisseria.gonorrhoeae, Original: Acinetobacter.baumanii
1/1 [==============================] - 0s 58ms/step
Predicted: Escherichia.coli, Original: Acinetobacter.baumanii
1/1 [==============================] - 0s 57ms/step
Predicted: Neisseria.gonorrhoeae, Original: Acinetobacter.baumanii
1/1 [==============================] - 0s 57ms/step
Predicted: Escherichia.coli, Original: Acinetobacter.baumanii
1/1 [==============================] - 0s 58ms/step
Predicted: Actinomyces.israeli, Original: Actinomyces.israeli
1/1 [==============================] - 0s 57ms/step
Predicted: Clostridium.perfringens, Original: Actinomyces.israeli
1/1 [==============================] - 0s 57ms/step
Predicted: Clostridium.perfringens, Original: Actinomyces.israeli
1/1 [==============================] - 0s 57ms/step
Predicted: A

1/1 [==============================] - 0s 59ms/step
Predicted: Propionibacterium.acnes, Original: Propionibacterium.acnes
1/1 [==============================] - 0s 59ms/step
Predicted: Propionibacterium.acnes, Original: Propionibacterium.acnes
1/1 [==============================] - 0s 57ms/step
Predicted: Propionibacterium.acnes, Original: Propionibacterium.acnes
1/1 [==============================] - 0s 56ms/step
Predicted: Propionibacterium.acnes, Original: Propionibacterium.acnes
1/1 [==============================] - 0s 57ms/step
Predicted: Escherichia.coli, Original: Proteus
1/1 [==============================] - 0s 59ms/step
Predicted: Escherichia.coli, Original: Proteus
1/1 [==============================] - 0s 58ms/step
Predicted: Escherichia.coli, Original: Proteus
1/1 [==============================] - 0s 56ms/step
Predicted: Escherichia.coli, Original: Proteus
1/1 [==============================] - 0s 57ms/step
Predicted: Escherichia.coli, Original: Proteus
1/1 [============